In [1]:
import os
import shutil

import numpy as np
import pandas as pd

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [2]:
data_directory = 'D:\\OneDrive - UW\\research\\data@aibs' # must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
manifest_path

'D:\\OneDrive - UW\\research\\data@aibs\\manifest.json'

In [4]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [5]:
sessions = cache.get_session_table()
#print('Total number of sessions: ' + str(len(sessions)))

#sessions.head()

In [6]:
probes = cache.get_probes()

#print('Total number of probes: ' + str(len(probes)))

#probes.head()

In [7]:

channels = cache.get_channels()

In [8]:

units = cache.get_units()

In [11]:
sessions[sessions.session_type == 'functional_connectivity']

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
766640955,2019-10-03T00:00:00Z,744912849,functional_connectivity,133.0,M,wt/wt,842,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, PF, TH, LP..."
767871931,2019-10-03T00:00:00Z,753795610,functional_connectivity,135.0,M,wt/wt,713,2231,6,"[MRN, APN, MB, DG, CA1, VISrl, nan, VISam, PO,..."
768515987,2019-10-03T00:00:00Z,754477358,functional_connectivity,136.0,M,wt/wt,802,2217,6,"[MB, APN, NOT, DG, CA1, VISam, nan, Eth, LP, P..."
771160300,2019-10-03T00:00:00Z,754488979,functional_connectivity,142.0,M,wt/wt,930,2230,6,"[SCig, PPT, NOT, SUB, ProS, CA1, VISam, nan, A..."
771990200,2019-10-03T00:00:00Z,756578435,functional_connectivity,108.0,M,wt/wt,546,2229,6,"[APN, MB, DG, CA1, VISam, nan, PO, LP, VISpm, ..."
774875821,2019-10-03T00:00:00Z,759711152,functional_connectivity,114.0,M,wt/wt,649,2233,6,"[APN, LP, DG, CA1, VISrl, nan, MGv, MGd, CA3, ..."
778240327,2019-10-03T00:00:00Z,760938797,functional_connectivity,120.0,M,wt/wt,784,2234,6,"[APN, POL, LP, DG, CA1, VISrl, nan, ZI, IntG, ..."
778998620,2019-10-03T00:00:00Z,759674770,functional_connectivity,121.0,M,wt/wt,793,2229,6,"[APN, POL, LP, DG, CA3, CA1, VISrl, nan, ZI, M..."
779839471,2019-10-03T00:00:00Z,760960653,functional_connectivity,122.0,M,wt/wt,863,2220,6,"[APN, LP, MB, DG, CA1, VISrl, nan, TH, LGd, CA..."


In [27]:
sessions.ecephys_structure_acronyms.values[44]

array(['PPT', 'MB', 'APN', 'NOT', 'HPF', 'ProS', 'CA1', 'VISam', nan,
       'POL', 'LP', 'DG', 'VISrl', 'ZI', 'IntG', 'IGL', 'CA3', 'VISal',
       'CA2', 'VISl', 'SUB', 'VISp', 'VISpm'], dtype=object)

In [12]:
data=cache.get_session_data(840012044)

D:\anaconda3\lib\site-packages\allensdk\api\caching_utilities.py:123: UserWarning: retrying fetch (1 tries remaining)
  warnings.warn(retry_message)


TimeoutError: 

In [29]:
data

In [30]:
data.probes

,description,location,sampling_rate,lfp_sampling_rate,has_lfp_data
id,,,,,
810758777,probeA,,29999.961312,1249.998388,True
810758779,probeB,,29999.913235,1249.996385,True
810758781,probeC,,29999.992168,1249.999674,True
810758783,probeD,,29999.916720,1249.996530,True
810758785,probeE,,29999.995551,1249.999815,True
810758787,probeF,,30000.034608,1250.001442,True


In [31]:
data.units

,isolation_distance,peak_channel_id,presence_ratio,waveform_spread,waveform_amplitude,silhouette_score,isi_violations,amplitude_cutoff,d_prime,waveform_velocity_below,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951082755,144.585054,849876486,0.99,40.0,236.404740,0.271803,0.020538,0.000057,8.940647,0.686767,...,10.0,SCig,8474,2739,7169,probeA,,29999.961312,1249.998388,True
951082761,87.025896,849876490,0.99,70.0,117.203385,0.204434,0.000000,0.003726,7.860537,-0.343384,...,10.0,SCig,8470,2729,7177,probeA,,29999.961312,1249.998388,True
951082759,88.072872,849876488,0.99,50.0,107.288220,0.221266,0.008631,0.001414,6.079214,0.000000,...,10.0,SCig,8472,2734,7173,probeA,,29999.961312,1249.998388,True
951082764,364.902781,849876492,0.99,50.0,89.019645,0.219987,0.002120,0.003792,6.028755,-1.030151,...,10.0,SCig,8468,2724,7181,probeA,,29999.961312,1249.998388,True
951082786,99.736844,849876500,0.99,60.0,164.961225,0.160511,0.000000,0.002400,9.705163,-0.206030,...,10.0,SCig,8459,2697,7200,probeA,,29999.961312,1249.998388,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951106447,50.611193,849871678,0.99,120.0,119.159004,0.055710,0.000000,0.024017,6.171032,0.515075,...,417.0,VISrl,7757,1001,8684,probeF,,30000.034608,1250.001442,True
951105767,117.489900,849871678,0.99,80.0,177.292635,0.182979,0.020093,0.001940,6.002761,0.000000,...,417.0,VISrl,7757,1001,8684,probeF,,30000.034608,1250.001442,True
951106361,77.255643,849871682,0.99,90.0,178.884810,NaN,0.000000,0.000084,8.236431,-0.343384,...,417.0,VISrl,7754,984,8690,probeF,,30000.034608,1250.001442,True


In [42]:
data.get_stimulus_table()

,Dir,Speed,coherence,color,contrast,frame,orientation,phase,pos,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,x_position,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,,,,,
0,null,null,null,null,null,null,null,null,null,null,null,8.931548,null,spontaneous,68.998348,null,null,null,60.066800,0
1,null,null,null,null,0.8,null,0,"[3644.93333333, 3644.93333333]","[-30.0, -10.0]","[20.0, 20.0]",0.08,68.998348,0,gabors,69.248554,4,-10,30,0.250206,1
2,null,null,null,null,0.8,null,90,"[3644.93333333, 3644.93333333]","[-30.0, -10.0]","[20.0, 20.0]",0.08,69.248554,0,gabors,69.498760,4,-20,-30,0.250206,2
3,null,null,null,null,0.8,null,90,"[3644.93333333, 3644.93333333]","[-30.0, -10.0]","[20.0, 20.0]",0.08,69.498760,0,gabors,69.732286,4,30,-10,0.233526,3
4,null,null,null,null,0.8,null,45,"[3644.93333333, 3644.93333333]","[-30.0, -10.0]","[20.0, 20.0]",0.08,69.732286,0,gabors,69.982492,4,0,0,0.250206,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77356,90,0.04,0.9,null,1,null,null,null,null,null,null,9126.569538,9,dot_motion,9127.553722,null,null,null,0.984183,2116
77357,45,0.02,0.9,null,1,null,null,null,null,null,null,9128.554568,9,dot_motion,9129.555402,null,null,null,1.000833,2113
77358,45,0.02,0.9,null,1,null,null,null,null,null,null,9130.556198,9,dot_motion,9131.557062,null,null,null,1.000863,2113


In [61]:
data.stimulus_names

['spontaneous',
 'gabors',
 'flashes',
 'drifting_gratings_contrast',
 'natural_movie_one_more_repeats',
 'natural_movie_one_shuffled',
 'drifting_gratings_75_repeats',
 'dot_motion']

In [45]:
data_dg=data.get_stimulus_table()[data.get_stimulus_table().stimulus_name=='drifting_gratings_contrast']

In [46]:
data_dg

,Dir,Speed,coherence,color,contrast,frame,orientation,phase,pos,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,x_position,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,,,,,
3798,null,null,null,null,0.35,null,135,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,1630.286968,2,drifting_gratings_contrast,1630.787380,2,null,null,0.500412,246
3799,null,null,null,null,1,null,0,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,1631.287792,2,drifting_gratings_contrast,1631.788205,2,null,null,0.500413,247
3800,null,null,null,null,0.08,null,135,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,1632.288618,2,drifting_gratings_contrast,1632.789040,2,null,null,0.500422,248
3801,null,null,null,null,0.04,null,0,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,1633.289462,2,drifting_gratings_contrast,1633.789880,2,null,null,0.500418,249
3802,null,null,null,null,0.6,null,135,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,1634.290298,2,drifting_gratings_contrast,1634.790720,2,null,null,0.500422,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,null,null,null,null,0.01,null,135,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,2165.734552,2,drifting_gratings_contrast,2166.234970,2,null,null,0.500418,251
4334,null,null,null,null,0.01,null,0,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,2166.735388,2,drifting_gratings_contrast,2167.235805,2,null,null,0.500417,271
4335,null,null,null,null,0.2,null,45,"[4198.96666667, 4198.96666667]","[0.0, 0.0]","[250.0, 250.0]",0.04,2167.736222,2,drifting_gratings_contrast,2168.236645,2,null,null,0.500423,267


In [35]:
data.spike_times

{951083985: array([5.33825146e+00, 5.47181830e+00, 5.50218500e+00, ...,
        9.81591354e+03, 9.81593267e+03, 9.81594284e+03]),
 951083957: array([4.79551743e+00, 7.39140732e+01, 7.51185081e+01, ...,
        9.51460011e+03, 9.51461245e+03, 9.68948841e+03]),
 951083886: array([  84.09111969,   84.09565302,   84.1038197 , ..., 9775.45375085,
        9781.65739218, 9781.74455896]),
 951083884: array([4.84975083e+00, 2.23217067e+01, 2.34154414e+01, ...,
        9.81594710e+03, 9.81611114e+03, 9.81615180e+03]),
 951083843: array([3.91554963e+00, 3.92201630e+00, 4.73335068e+00, ...,
        9.81461800e+03, 9.81542170e+03, 9.81592940e+03]),
 951083838: array([5.32098477e+00, 5.58978512e+00, 3.35467878e+01, ...,
        9.81167313e+03, 9.81477883e+03, 9.81480043e+03]),
 951083821: array([  68.26979928,   98.13207113,   98.47117156, ..., 9806.10225704,
        9815.42906907, 9815.83733626]),
 951083801: array([4.89488422e+00, 5.07081778e+00, 6.26531932e+00, ...,
        9.81488490e+03, 9.8153

In [39]:
data.units.index

Int64Index([951082755, 951082761, 951082759, 951082764, 951082786, 951082795,
            951082791, 951082798, 951082828, 951082838,
            ...
            951105734, 951106350, 951106341, 951106335, 951105779, 951106447,
            951105767, 951106361, 951105795, 951105928],
           dtype='int64', name='unit_id', length=1005)

In [40]:
data.spike_times[data.units.index[0]]

array([3.69361601e+00, 3.87444957e+00, 3.90548295e+00, ...,
       9.81352760e+03, 9.81353403e+03, 9.81598637e+03])

In [49]:
unit_spikes=data.spike_times[data.units.index[0]]

In [60]:
unit_spikes[np.logical_and(unit_spikes>=data_dg.start_time.values[0],unit_spikes<data_dg.start_time.values[1])]

array([1630.29488031, 1630.58018068, 1630.6447141 , 1630.67111413,
       1630.68534748, 1630.6990475 , 1630.71088085, 1630.7281142 ,
       1630.7481809 ])

In [13]:
60/16

3.75

In [14]:
-51.75*(4/15)

-13.799999999999999

In [15]:
2/13.8

0.14492753623188406

In [16]:
23/6

3.8333333333333335